In [1]:
WORK_DIR_PATH = ".."

In [2]:
GROUP_MAP = {
    '외식업': ['한식음식점','분식전문점','호프-간이주점','제과점','커피-음료','일식음식점','패스트푸드점','양식음식점','중식음식점','치킨전문점','반찬가게'],
    '필수재소매업': ['편의점','슈퍼마켓','육류판매','청과상','미곡판매','수산물판매','화장품'],
    '사치재소매업': ['문구','철물점','운동/경기용품','가방','신발','완구','섬유제품','시계및귀금속','인테리어','가구','조명용품','화초','일반의류','서적'],
    '건강소매업': ['의약품','의료기기'],
    '생활서비스업': ['미용실','네일숍','세탁소','안경','피부관리실','애완동물'],
    '교육서비스업': ['일반교습학원','예술학원','외국어학원'],
    '건강서비스업': ['일반의원','치과의원','한의원'],
    '자동차업': ['자동차수리','자동차미용','자전거 및 기타운송장비'],
    '가전제품업': ['컴퓨터및주변장치판매','가전제품','가전제품수리','핸드폰'],
    '숙박업': ['고시원','여관'],
    '부동산업': ['부동산중개업'],
    '실내여가업': ['스포츠 강습','스포츠클럽','PC방','노래방','당구장','골프연습장'],
    '전자상거래업': ['전자상거래업'],
}

# Packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.chdir(WORK_DIR_PATH)
print(f"DIRECTORY: {os.getcwd()}")

DIRECTORY: c:\Users\jayar\Desktop\바탕 화면\REPO\PROJECT\M2-PJT_STATS


In [5]:
import numpy as np
import pandas as pd

# Functions

In [6]:
def df_converter(df):
    reverse_map = {v: k for k, values in GROUP_MAP.items() for v in values}
    df['서비스_업종_그룹_명'] = df['서비스_업종_코드_명'].map(reverse_map)

    df_grouped = (
        df.groupby(
            by=['기준_년분기_코드','상권_구분_코드_명','서비스_업종_그룹_명'],
            as_index=False,
        )[['당월_매출_건수','당월_매출_금액']]
        .sum()
    )
    
    return df_grouped

In [ ]:
def post_discriminator(df):
    df["YEAR"] = df['기준_년분기_코드'].astype(str).str[:4].astype(int)
    df["QUARTER"] = df['기준_년분기_코드'].astype(str).str[-1].astype(int)

    df['POST'] = 0

    # DURING COVID
    CONDITION = (
        ((df['YEAR'] == 2020) & (df['QUARTER'].isin([2,3,4]))) 
        | (df['YEAR'] == 2021) 
        | ((df['YEAR'] == 2022) & (df['QUARTER']==1))
    )
    df.loc[CONDITION, 'POST'] = 0

    # POST-COVID
    CONDITION = (
        ((df['YEAR'] == 2022) & (df['QUARTER'].isin([2,3,4]))) 
        | (df['YEAR'] == 2023)
        | (df['YEAR'] == 2024) & (df['QUARTER']==1)
    )
    df.loc[CONDITION, 'POST'] = 1

    df['TIME'] = df['기준_년분기_코드'].astype('category').cat.codes
    df = df.drop(columns=['기준_년분기_코드'])

    return df

# Process

In [8]:
df_list = []

for YEAR in [2020, 2021, 2022, 2023, 2024]:
    df = pd.read_csv(f"./_data/origin/{YEAR}.csv", encoding="cp949")
    if YEAR==2020:
        CONDITION = df["기준_년분기_코드"]!=20201
        df = df[CONDITION]
    if YEAR==2024:
        CONDITION = df["기준_년분기_코드"]==20241
        df = df[CONDITION]
    df_grouped = df_converter(df)
    df_list.append(df_grouped)

df_merged = pd.concat(objs=df_list, ignore_index=True)

In [ ]:
df_discriminated = post_discriminator(df_merged)

In [ ]:
RE_COL = dict(
    상권_구분_코드_명="AREA",
    서비스_업종_그룹_명="INDUSTRY",
    당월_매출_건수="COUNT",
    당월_매출_금액="AMOUNT",
)

df_renamed = df_discriminated.rename(columns=RE_COL)

In [11]:
df_renamed['LOG_COUNT'] = np.log1p(df_renamed['COUNT'])
df_renamed['LOG_AMOUNT'] = np.log1p(df_renamed['AMOUNT'])

In [12]:
SORTED_COL = [
    "TIME","YEAR","QUARTER",
    "POST",
    "AREA","INDUSTRY",
    "COUNT","AMOUNT","LOG_COUNT","LOG_AMOUNT"
]

df_sorted = df_renamed[SORTED_COL]

# Save

In [13]:
df_sorted.head()

,TIME,YEAR,QUARTER,POST,AREA,INDUSTRY,COUNT,AMOUNT,LOG_COUNT,LOG_AMOUNT
0,0,2020,2,0,골목상권,가전제품업,13016879,96975914913,16.381758,25.297728
1,0,2020,2,0,골목상권,건강서비스업,6679518,182430757093,15.714557,25.929637
2,0,2020,2,0,골목상권,건강소매업,16410010,360977224528,16.613402,26.612081
3,0,2020,2,0,골목상권,교육서비스업,451345,132232285601,13.019990,25.607826
4,0,2020,2,0,골목상권,부동산업,29816,1183998884,10.302834,20.892163


In [14]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TIME        828 non-null    int8   
 1   YEAR        828 non-null    int64  
 2   QUARTER     828 non-null    int64  
 3   POST        828 non-null    int64  
 4   AREA        828 non-null    object 
 5   INDUSTRY    828 non-null    object 
 6   COUNT       828 non-null    int64  
 7   AMOUNT      828 non-null    int64  
 8   LOG_COUNT   828 non-null    float64
 9   LOG_AMOUNT  828 non-null    float64
dtypes: float64(2), int64(5), int8(1), object(2)
memory usage: 59.2+ KB


In [15]:
df_sorted.isna().sum()

TIME          0
YEAR          0
QUARTER       0
POST          0
AREA          0
INDUSTRY      0
COUNT         0
AMOUNT        0
LOG_COUNT     0
LOG_AMOUNT    0
dtype: int64

In [16]:
df_sorted.to_csv("./_data/sales.csv", index=False)